In [36]:
import os

from llama_index import Document, GPTSimpleVectorIndex, ServiceContext, SimpleDirectoryReader
from llama_index.data_structs.node_v2 import DocumentRelationship
from llama_index.response.schema import RESPONSE_TYPE
from llama_index import LLMPredictor, PromptHelper
from langchain.chat_models import ChatOpenAI
import json
import glob
from LLMSearch.TextSplitter import split_text

import re

In [50]:
with open('settings/settings.json') as f:
    settings = json.load(f)

original_folder_path=settings["data_path"]+"/original"
split_folder_path=settings["data_path"]+"/split"
chunk_size_limit=settings["chunk_size_limit"]

def filename_fn(filename): return {"file_name": filename}

directory_reader = SimpleDirectoryReader(
    original_folder_path,
    file_metadata=filename_fn,
    recursive=True,
)

available_file_list= [str(input_file)
                for input_file in directory_reader.input_files]   

#check for finished documents
split_file_list=glob.glob(split_folder_path+"/*")
fin_file_list=[re.sub(r"_[0-9]+.txt","",file) for file in split_file_list]
fin_file_list=list(set(fin_file_list))
fin_file_list=[os.path.basename(file) for file in fin_file_list]
fin_file_list=[original_folder_path+"/"+file for file in fin_file_list]

fin_ids=[]
for i,file in enumerate(available_file_list):
    if file in fin_file_list:
        fin_ids.append(i)

for i in sorted(fin_ids, reverse=True):
        directory_reader.input_files.pop(i)


documents = directory_reader.load_data()

for doc in documents:
    text=doc.text
    file_path=doc.extra_info["file_name"]
    file_name=os.path.basename(file_path)

    chunk_list=split_text(text,chunk_size_limit)

    for i,chunk in enumerate(chunk_list):
        output_file=split_folder_path+"/"+file_name+"_"+str(i)+".txt"
        with open(output_file, 'w') as f:
            f.write(chunk)

In [51]:
fin_ids

[0, 1, 2]

In [52]:
documents

[]

In [56]:
from LLMSearch.DocSplitter import split_documents

split_documents()


skip: UserData/original/ginga_tets.txt
skip: UserData/original/wagahai.txt
split: sikkaku.pdf
